# <center>HARRY POTTER</center>

## <center>and the Text Mining Project</center>

#### <center>Enrico Carraro, Alex Cecchetto, Virginia Murru</center>

1. [Data](#data)<br>
2. [Exploratory Data Analysis](#eda)<br>
    2.1 [Bi-Grams](#bigrams)<br>
3. [Harry Potter and the order of the books](#order)<br>
    3.1 [Formulation of the problem](#problem)<br>
    3.2 [t-test](#ttest)<br>
    3.3 [Harry Potter and the selected words](#words)<br>
4. [Similarity between books](#sim)<br>
    4.1 [K-means](#km)<br>
    4.2 [Spectral Clustering](#spec)<br>  
5. [Sentiment analysis](#sent)<br>

In [1]:
# import the libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
# from collections import Counter
from wordcloud import WordCloud
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer # Sentiment analysis
from nrclex import NRCLex

## 1. Data <a id=data> </a>

In [2]:
# import the books
from Functions.processing import read_hp

book1 = read_hp('Data/Book1.txt')
book2 = read_hp('Data/Book2.txt')
book3 = read_hp('Data/Book3.txt')
book4 = read_hp('Data/Book4.txt')
book5 = read_hp('Data/Book5.txt')
book6 = read_hp('Data/Book6.txt')
book7 = read_hp('Data/Book7.txt')

In [3]:
# Collect all the books in a unique object and transform the text to the lower case
books = book1 + book2 + book3 + book4 + book5 + book6 + book7
books = books.lower()

The first thing to do is to preprocess the data:
1. create a custom tokenizer in order to obtain a collection of all the words;
2. remove the stop_words, using a collection of common words that has no meaning for the analysis;
3. reduce the tokens to the stems, that is part of a word, that is common to all of its inflected variants

In [4]:
from Functions.processing import CustomTokenizer
# the costum Tokenizer has been made in order to collect only the words and without considering abbreviation 
# (like the possessive case) and the punctuation

custom_tokenizer = CustomTokenizer()
hp_tokens = custom_tokenizer.tokenize(books)
hp_tokens = [i[0] for i in hp_tokens if i[1] == 'WORD'] # this is going to collect only the words

In [5]:
# remove the stop words from the collection of token
with open("Data/stop_words.txt", 'r') as file:
    hp_stop_w = [words.strip() for words in file.readlines() if not (words.startswith("Page |") or words.strip() == '')]

hp_tokens_sw = [i for i in hp_tokens if i not in hp_stop_w]
# remove the following symbols
hp_tokens_sw = [word for word in hp_tokens_sw if not word.startswith("'")] 

## 2. Exploratory Data Analysis <a id=eda> </a>

In [ ]:
#function that counts the frequency of each token and retrieve the most frequent N tokens
from Functions.processing import counter
hp_counter = counter(hp_tokens_sw)

In [ ]:
N = 45
plt.figure(figsize=(15, 3))

# plt.subplot(121)
plt.title("{} Most frequent words in the Harry Potter series".format(N))
plt.bar(*zip(*hp_counter.most_common(N)), color="gold")
plt.xticks(rotation="vertical")



plt.show()

In [ ]:
# word cloud of the most frequent tokens
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(" ".join(hp_tokens_sw))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# custom wordcloud
from PIL import Image
MASK = np.array(Image.open("Images/Sorting_Hat.png"))
MAX_WORDS = 300
MAX_FONT_SIZE = 500
RELATIVE_SCALING = 0.7

hp = WordCloud(
    width = 2000, 
    height = 2000,
    mask = MASK,
    max_words = MAX_WORDS, 
    background_color = "white",
    max_font_size = MAX_FONT_SIZE,
    relative_scaling = RELATIVE_SCALING,
).generate_from_frequencies(hp_counter.counts)

plt.figure(figsize=(20, 10))
plt.imshow(hp, interpolation='bilinear')
plt.axis('off')
plt.show()

### 2.1 BI-GRAMS <a id=bigrams>

In [ ]:
bigrm = nltk.bigrams(hp_tokens_sw)
bigram_fd = nltk.FreqDist(bigrm)
bigram_fd.most_common(10)

bigram_freq = {f'{key[0]} {key[1]}': value for key, value in dict(bigram_fd).items()}

In [ ]:
from PIL import Image
MASK = np.array(Image.open("Images/harry.png"))
MAX_WORDS = 500
MAX_FONT_SIZE = 500
RELATIVE_SCALING = 0.7

hp = WordCloud(
    width = 1000, 
    height = 1000,
    mask = MASK,
    max_words = MAX_WORDS, 
    background_color = "white",
    max_font_size = MAX_FONT_SIZE,
    relative_scaling = RELATIVE_SCALING,
).generate_from_frequencies(bigram_freq)

plt.figure(figsize=(30, 15))
plt.imshow(hp, interpolation='bilinear')
plt.axis('off')
plt.show()


In [ ]:
# obtain the stems from the collection of books
stemmer = SnowballStemmer("english")
hp_stem = [stemmer.stem(token) for token in hp_tokens_sw]

### CREATION OF DICTIONARIES BY BOOK

In [ ]:
hp_books = [book1, book2, book3, book4, book5, book6, book7]

In [ ]:
from Functions.processing import preproc

books_hp_token = {}
for i in range(len(hp_books)):
    book_name = f"book_{i+1}"
    books_hp_token[book_name] = preproc(hp_books[i], custom_tokenizer, stop_words = hp_stop_w)

books_hp_stem = {}
for i in range(len(hp_books)):
    book_name = f"book_{i+1}"
    books_hp_stem[book_name] = [stemmer.stem(token) for token in books_hp_token[book_name]]

## 3. Harry Potter and the order of the books <a id=order>

The goal is to reconstruct the chronological order of the seven books, starting from the analysis of the texts. The idea is to assume that the position of the first and the last book, classificating the books from 2 to 6 only using the words available and thus, analyzing the evolution of the narrative style. 

To do this, we considered the most frequent words available in both of the starting books, keeping only the ones that are present also in the other 5 books. 

In [ ]:
# order the dictionary by the frequence of each stem
book11 = counter(books_hp_stem['book_1']).most_common()
book77 = counter(books_hp_stem['book_7']).most_common()

In [ ]:
# take the most 400 common stems that are in book 1 and book 7
book11 = dict(list(book11.items())[:400])
book77 = dict(list(book77.items())[:400])

# keep only the common strems
most_comm_17 = list(set(book11.keys()) & set(book77.keys()))

In [ ]:
most_comm_tot = most_comm_17.copy()

In [ ]:
# from the list of the most common stems, keep the one that are also in the other books
for i in range(len(books_hp_stem)):
    book_name = f"book_{i+1}"
    most_comm_tot = [i for i in most_comm_tot if i in books_hp_stem[book_name]]

### 3.1 Formulation of the Problem <a id=problem>

We consider two initial population:
1. $W_1$ = *Harry Potter and the Phylosopher stone*
2. $W_7$ = *Harry Potter and the deathly hallows*.

Let, for *j* = {1, 7}:
- $n_{ij}$ be the frequence of the $i$-$th$ stem for $i = 1, \dots, N$ in book $j$, with $N$ the total number of stems considered;
- $N_j = \sum_{i}n_{ij}$
- $\theta_{ji}$, for $i = 1, \dots, N$ the probability that the i-th word is in $W_j$.

The probability to observe a sample from $W_1$ and $W_7$ follows a multinomial distribution. Using the log likelihood ratio test to compare the two samples we obtain:

$$ \sum_{i=1}^{N} n_i\log\frac{\theta_{7i}}{\theta_{1i}}$$

We can see that every word has a sort of score, like:

$$ s_i = \log{\frac{\theta_{7i}}{\theta_{1i}}}$$



In [ ]:
filtered_dict = {key: value for key, value in counter(books_hp_stem['book_1']).counts.items() if key in most_comm_tot}
xx = sorted(filtered_dict.items(),key=lambda item: item[1], reverse=True)
df_book = pd.DataFrame(xx, columns = ['Stems', 'Count_1'])
df_book['Freq_1'] = df_book['Count_1']/df_book['Count_1'].sum()
df_book.head()

filtered_dict = {key: value for key, value in counter(books_hp_stem['book_7']).counts.items() if key in most_comm_tot}
xx = dict(sorted(filtered_dict.items(),key=lambda item: item[1], reverse=True))
df_book['Count_7'] = df_book['Stems'].map(xx)
df_book['Freq_7'] = df_book['Stems'].map(xx)/df_book['Stems'].map(xx).sum()
df_book.head()


The idea is to extend the analysis to the other books, which represent new populations to be classified, and use the total score of each book as a discriminant function. Thus let $W_k$ with $k = 2,3,4,5,6$ be the populations that represent the books from 2 to 6. For these populations, the same results apply as for $W_1$ to $W_7$. So for each $W_k$ with $k = 2,3,4,5,6,7$ we can calculate a measure that represents the comprehensive score assigned to each $W_k$. In particular for each $W_k$ where $N_k = \sum_i n_{ik}$ one can calculate the average score

$$
\bar{s}_k = \frac{1}{N_k} \sum_{i=1}^{N} n_{i} \log \frac{\theta_{1i}}{\theta_{7i}} = \sum_{i=1}^{N} n_{i}s_i
$$

To calculate the average scores we need to estimate the parameter vectors $\theta_1$ and $\theta_7$. Since both $N_1$ and $N_7$ are high, the probabilities $\hat{\theta}_{i1}$ and $\hat{\theta}_{i7}$ can be estimated with the corresponding observed frequencies. So we find for all five books to be classified an associated score that represents the positioning of that book. This measure can be interpreted in relative terms to understand which are the furthest books and which are the closest. To make inferences and evaluate the significance of the results obtained, we can also define the variance of $s_k$, for which an unbiased estimate is


$$
\hat{V}(\bar{s}_k) = \frac{1}{N_k(N_k - 1)} \left( \sum_{i=1}^{N} n_i s_i^2 - \frac{1}{N_k} \left( \sum_{i=1}^{N} n_i s_i \right)^2 \right).
$$

Given that $N_k$ is large, $s_k - s_k'$ will be approximately normally distributed with variance equal to the sum of the corresponding variances; therefore, to test the significance, we can use the usual $t$-test.


In [ ]:
from Functions.ordering import hp_count

# with this loop we add a column for each of the book with their own freq and counts
for i in range(2, 7):
    book_idx = f"book_{i}"
    col_count, col_freq = hp_count(book_idx, books_hp_stem, most_comm_tot, df_book)
    df_book[f"Count_{i}"] = np.array(col_count)
    df_book[f"Freq_{i}"]= np.array(col_freq)

df_book.head()

In [ ]:
from Functions.ordering import hp_scores

scores = {}
for i in range(2, 7):
    scores[f"book_{i}"] = {'score': hp_scores(df_book, i)}

sorted_scores = sorted(scores.items(), key=lambda item: item[1]['score'], reverse = True)
scores = {k: v for k, v in sorted_scores}

scores

In [ ]:
from Functions.ordering import hp_var
for i in range(2, 7):
    scores[f"book_{i}"]['Variance'] = hp_var(df_book, i)
    scores[f"book_{i}"]['N'] = sum(df_book[f"Count_{i}"])
scores

### 3.2 t-test <a id=ttest>

In [ ]:
# from scipy import stats
from Functions.ordering import t_test

ttest = {}
for i in range(2,7):
    for j in range(i+1,7):
        ttest[f"test_{i}_{j}"] = {'t-stat': round(t_test(scores[f"book_{i}"], scores[f"book_{j}"])[0],3), 
                              'pval': round(t_test(scores[f"book_{i}"], scores[f"book_{j}"])[1], 3)}
ttest
        

The books do not seem to follow a coherent order. Probably, the chosen set of words is very generic, and considering the vastness of the analyzed texts, specific lexical constructs are captured. However, these constructs are not necessarily related to the plot and style of the various books but rather result from the large size of the books.

Therefore, an attempt has been made to select words using some form of criteria. In particular, ontological dictionaries were used to identify words that could indicate negative sentiments.

### 3.3 Harry Potter and the selected words <a id=words>

In [ ]:
# Create an instance of NRCLex
text_object = NRCLex(book1)

# book1
emot_1 = text_object.affect_dict
set(text_object.affect_list)
filter1 = [key for key, value in emot_1.items() if 'anger' in value or 'fear' in value 
                 or 'negative' in value or 'sadness' in value]

# book7
text_object = NRCLex(book7)
emot_7 = text_object.affect_dict
# set(text_object.affect_list)
filter7 = [key for key, value in emot_7.items() if 'anger' in value or 'fear' in value 
                 or 'negative' in value or 'sadness' in value]

In [ ]:
most_comm_filt = list(set(filter1) & set(filter7))
# obtain the stems 
most_comm = [stemmer.stem(token) for token in most_comm_tot]
most_comm_tot = most_comm.copy()
for i in range(len(books_hp_stem)):
    book_name = f"book_{i+1}"
    most_comm_tot = [i for i in most_comm_tot if i in books_hp_stem[book_name]]

In [ ]:
len(most_comm_tot)

In [ ]:
#filtered_dict = {key: value for key, value in counter(books_hp_stem['book_1']).most_common(300).items() if key in most_comm_tot}
filtered_dict = {key: value for key, value in counter(books_hp_stem['book_1']).counts.items() if key in most_comm_tot}
xx = sorted(filtered_dict.items(),key=lambda item: item[1], reverse=True)
df_book = pd.DataFrame(xx, columns = ['Stems', 'Count_1'])
df_book['Freq_1'] = df_book['Count_1']/df_book['Count_1'].sum()
# df_book.head()

filtered_dict = {key: value for key, value in counter(books_hp_stem['book_7']).counts.items() if key in most_comm_tot}
xx = dict(sorted(filtered_dict.items(),key=lambda item: item[1], reverse=True))
df_book['Count_7'] = df_book['Stems'].map(xx)
df_book['Freq_7'] = df_book['Stems'].map(xx)/df_book['Stems'].map(xx).sum()
# df_book.head()

for i in range(2, 7):
    book_idx = f"book_{i}"
    col_count, col_freq = hp_count(book_idx, books_hp_stem, most_comm_tot, df_book)
    df_book[f"Count_{i}"] = np.array(col_count)
    df_book[f"Freq_{i}"]= np.array(col_freq)

# df_book.head()

#from func.proc import hp_scores

scores = {}
for i in range(2, 7):
    scores[f"book_{i}"] = {'score': hp_scores(df_book, i)}
#aggiunta
    

sorted_scores = sorted(scores.items(), key=lambda item: item[1]['score'], reverse = True)
scores = {k: v for k, v in sorted_scores}

for i in range(2, 7):
    scores[f"book_{i}"]['Variance'] = hp_var(df_book, i)
    scores[f"book_{i}"]['N'] = sum(df_book[f"Count_{i}"])
scores

In [ ]:
ttest = {}
for i in range(2,7):
    for j in range(i+1,7):
        ttest[f"test_{i}_{j}"] = {'t-stat': round(t_test(scores[f"book_{i}"], scores[f"book_{j}"])[0],3), 
                              'pval': round(t_test(scores[f"book_{i}"], scores[f"book_{j}"])[1], 3)}
ttest

## 4. Similarity between books <a id=sim>

We start from the matrix of stems, where for each row we have the frequency of each stem within the individual book:

In [ ]:
documents = [" ".join(words) for words in books_hp_stem.values()]
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(documents)

count_array = count_matrix.toarray()

freq_matrix = np.divide(count_array, np.sum(count_array, axis=1, keepdims=True))

In [ ]:
np.sum(count_array, axis = 1)

### 4.1 k-means <a id=km>

We use a custom kmeans to cluster the seven books. In particular we chose as centroids the first and the last book.

In [ ]:
from Functions.clustering import cosine, centroidi, kmeans

k_centroids = count_array[[0,6]]

In [ ]:
sorted_array = count_array.copy()
np.random.shuffle(sorted_array)
gruppi_KM = kmeans(sorted_array, k_centroids)
gruppi_KM

In [ ]:
#going back to the original order based on the number of stems of each vectr
initial_mat = np.sum(count_array, axis = 1)
sorted_mat = np.sum(sorted_array, axis = 1)
sorted_indices_array1 = np.argsort(initial_mat)
positions_in_array1 = sorted_indices_array1[np.argsort(np.argsort(sorted_mat))]


In [ ]:
clusters = {}
for i in gruppi_KM:
    clusters[f"cluster_{i + 1}"] = [f"book_{i}" for i in positions_in_array1[gruppi_KM[i]] + 1]
clusters

### 4.2 Spectral Clustering <a id=spec>

To perform spectral clustering we first have to represent data as a graph, with vertices and edges, represented in the form $\mathcal{G} = \{V, E\}$. To do so an intuitive way is to use as vertices all the observation (books) and as weights to connect them the distance between each couple of observation. In particular we will use the cosine distance $d(w_i, w_j)$ already defined. In this way we get that all the vertices are connected with weights given by the inputs of matrix W, where
$$
W = w_{ij}\quad\text{with}\quad w_{ij} = d(v_i, v_j).
$$
Moreover, we need to define the degree matrix 
$$
D = diag(d_i)
$$ 
with all empty off-diagonal entries, whereas the diagonal contains the degree of each node, which is the number of edges incident on it. We will refer to it as 
$$
d_i = \sum_{j = 1}^{n}w_{ij}.
$$

It is now necessary to define the Graph Laplacian $L = D - W$ and normalize it as folows: 
$$
L_{\text{sym}}:=D^{-\frac{1}{2}}LD^{-\frac{1}{2}} = I - D^{-\frac{1}{2}}WD^{-\frac{1}{2}}
$$

### Normalized Spectral clustering according to Ng, Jordan, and Weiss (2002)
Input: weight matrix $W\in \mathbb{R}^{n\times n}$, number $k$ of clusters to construct
* Compute the normalized Laplacian $L_{\text{sym}}$.
* Compute the first $k$ eigenvectors $u_1, \dots , u_k$ of $L_{\text{sym}}$ correspondent to the smallest $k$ eigenvalues.
* Let $U\in \mathbb{R}^{n\times k}$ be the matrix containing the vectors $u_1, \dots , u_k$ of $L_{\text{sym}}$ as columns.
* Form the matrix $T\in \mathbb{R}^{n\times k}$ from $U$ by normalizing the rows to norm 1, that is set $t_{ij} = u_{ij}/(\sum_k u_{ik}^2)^{1/2}$
* For $i = 1, \dots, n$, let $y_i \in\mathbb(R){k}$ be the vector corresponding to the $i$-th row of $T$.
* Cluster the points $(y_i)_{i = 1, \dots, n}$ with the $k$-means algorithm into clusters $C_1, \dots, C_k$.

Output: Clusters $A_1, \dots, A_k$ with $A_i = \{j\,|\,y_j\in C_i\}$.

In [ ]:
from Functions.clustering import spectral_cl

In [ ]:
spectral_cl(count_array, 2)

## 5. Sentiment analysis <a id=sent>

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
book_names = list()
for i in range(len(hp_books)): 
    book_names.append(f"book_{i+1}")
print(book_names)

In [ ]:
scores = {}
for b in book_names:
    new_b = " ".join(books_hp_token[b])
    scores[b] = analyzer.polarity_scores(new_b)
    print(b)

In [ ]:
scores

In [ ]:
sorted_scores = dict(sorted(scores.items(), key = lambda item: item[1]["neg"]))

for key in sorted_scores:
    print(key + ": ")
    print(sorted_scores[key])
    print()

In [ ]:
sorted_scores = dict(sorted(scores.items(), key=lambda item: item[1]["neu"]))
for key in sorted_scores:
    print(key + ": ")
    print(sorted_scores[key])
    print()

As we can see the books are perfectly ordered for the negative score, which means that in each book, even though there isn't a worse feeling with year after year, we can find a growing presence of you know who and of the death eaters, sometimes compensated by positive feelings and experiences of the main characters

In [ ]:
book_emotions = dict()
keys = ["fear", "anger", "anticip", "trust", "surprise", "positive", "negative", "sadness", "disgust", "joy", "anticipation"]
for b in book_names:
    print(b)
    emotion = dict()
    for k in keys:
        emotion[k] = 0    
    for t in books_hp_token[b]:
        e = NRCLex(t).affect_frequencies
        for k in e.keys():
            emotion[k] += e[k]
    book_emotions[b] = emotion
print("done")

In [ ]:
em_stand = dict()
for k1, e in book_emotions.items():
    em_stand[k1] = dict()
    tot_em = sum(e.values())
    for k2 in e.keys():
        if k2 != "anticip":
            em_stand[k1][k2] = e[k2] / tot_em
            
print(em_stand["book_1"].keys())

In [ ]:
# trust, disgust
# watch out: negative doesn't give such a good order as with vader
sorted_emotions = dict(sorted(em_stand.items(), key=lambda item: item[1]["trust"]))
print(sorted_emotions.keys())

In [ ]:
print(em_stand)

In [ ]:
from collections import Counter

counts=Counter(hp_tokens_sw)

In [ ]:
sent = {}
for b, ws in books_hp_token.items():
    for w in ws:
        a = analyzer.polarity_scores(w)
        for k in a:
            if  a[k] > 0:
                sent[w] = sent.get(w, set())
                sent[w].add(k)
    


for i in sent:
    sent[i] = list(sent[i])

print(sent)

In [ ]:
book_names

In [ ]:
sent_book = {}
for b in book_names:
    sent = {}
    for w in books_hp_token[b]:
        a = analyzer.polarity_scores(w)
        for k in a:
            if  a[k] > 0:
                sent[w] = sent.get(w, set())
                sent[w].add(k)
    for i in sent:
        sent[i] = list(sent[i])
    sent_book[b] = sent
    




print(sent_book)

In [ ]:
counts=Counter(hp_tokens_sw)

word_counts=dict()
for i in sent.keys():
  if ("pos" in sent[i]): word_counts[i]=counts[i]


surprise=pd.DataFrame.from_dict(word_counts, orient='index')
surprise = surprise.reset_index()
surprise = surprise.rename(columns={'index' : 'words' , 0: 'count'})
surprise=surprise.sort_values("count",ascending=True)

plt.figure(figsize=(10, 5))
plt.barh(surprise['words'][(len(surprise)-15):], surprise['count'][(len(surprise)-15):], color="#E5BE01")
#sns.barplot(x="words",y="count", data=anger[50:])
plt.title("Most common positive words")
plt.xlabel('Frequencies')
plt.ylabel('Words')
plt.show()

In [ ]:
word_counts=dict()
for i in sent.keys():
  if ("neg" in sent[i]): word_counts[i]=counts[i]


surprise=pd.DataFrame.from_dict(word_counts, orient='index')
surprise = surprise.reset_index()
surprise = surprise.rename(columns={'index' : 'words' , 0: 'count'})
surprise=surprise.sort_values("count",ascending=True)

plt.figure(figsize=(10, 5))
plt.barh(surprise['words'][(len(surprise)-15):], surprise['count'][(len(surprise)-15):], color="#003399")
#sns.barplot(x="words",y="count", data=anger[50:])
plt.title("Most common negative words")
plt.xlabel('Frequencies')
plt.ylabel('Words')
plt.show()